In [36]:
#Start Redis and MySQL servers
!docker compose up -d

[+] Running 0/0
 ⠋ Network ipvo_default  Creating                                          0.1s 
[+] Running 1/1
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠋ Container ipvo-db-1     Creating                                        0.0s 
 ⠋ Container redis-server  Creating                                        0.0s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠿ Container ipvo-db-1     Starting                                        0.1s 
 ⠿ Container redis-server  Starting                                        0.1s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠿ Container ipvo-db-1     Starting                                        0.2s 
 ⠿ Container redis-server  Starting                                        0.2s 
[+] Running 1/3
 ✔ Network ipvo_default    Created                                         0.1s 
 ⠿ Container ipvo-db-1     St

In [37]:
#Connecting to the MySQL server
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="user",
  password="password",
  database="db"
)

mycursor = mydb.cursor()

In [38]:
#Connecting to the Redis server
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [39]:
#Importing Faker which is used for generating fake data, we need this because we're benchmarking writes
from faker import Faker
Faker.seed(123)
fake = Faker()

TypeError: Calling `.seed()` on instances is deprecated. Use the class method `Faker.seed()` instead.

In [ ]:
#Setting up how many writes/reads we want, 11 means we'll get 10 writes/reads
repeat = 11

In [ ]:
#Checking if everything works on a small amount of data. This block writes data into cache(Redis) and the db(MySQL)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    print(name)
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)

Michael Baker
Vincent Martinez
Jeffery Hines
Derek Nunez
Karen Carney
Audrey Smith
Daniel Hill
Jose Henderson
Thomas Marsh
Scott Clark


In [ ]:
#Reading the data in cache and db to see if they were succesfully written
for key in range(1, repeat):
    temp = r.get(key)
    print(temp)
    sql = "SELECT value FROM tablica WHERE id = %s"
    tpl = (key, )
    mycursor.execute(sql, tpl)
    myresult = mycursor.fetchall()
    print(myresult)

Michael Baker
[('Michael Baker',)]
Vincent Martinez
[('Vincent Martinez',)]
Jeffery Hines
[('Jeffery Hines',)]
Derek Nunez
[('Derek Nunez',)]
Karen Carney
[('Karen Carney',)]
Audrey Smith
[('Audrey Smith',)]
Daniel Hill
[('Daniel Hill',)]
Jose Henderson
[('Jose Henderson',)]
Thomas Marsh
[('Thomas Marsh',)]
Scott Clark
[('Scott Clark',)]


In [ ]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [ ]:
#Importing time, we need it to time how long it takes to execute writes/reads
import time

In [ ]:
#Setting repeat to the number we want to benchmark writes with
repeat = 101

In [ ]:
#This block benchmarks how long it takes to write data if we don't have a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
t0 = time.time()
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t1 = time.time()
print("Time it took to write: ", t1-t0)

Time it took to write:  0.07177853584289551


In [ ]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [ ]:
#This block benchmarks how long it takes to write data if we don't have a cache 100 times and calculates the average write time
ta = 0
for i in range(100):
    mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
    t0 = time.time()
    for key in range(1, repeat):
        name = fake.name()
        sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
        val = (key, name)
        mycursor.execute(sql, val)
    t1 = time.time()
    ta = ta + (t1 - t0)
    mycursor.execute("DROP TABLE tablica")
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)

Average write time after repeating the benchmark 100 times is:  0.21080610513687134


In [ ]:
#This block benchmarks how long it takes to write data if we have a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
t0 = time.time()
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t1 = time.time()
print("Time it took to write: ", t1-t0)

Time it took to write:  0.3403632640838623


In [ ]:
#Reseting cache and db for the benchmark
r.flushall()
mycursor.execute("DROP TABLE tablica")

In [ ]:
#This block benchmarks how long it takes to write data if we have a cache 100 times and calculates the average write time
ta = 0
for i in range(100):
    mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
    t0 = time.time()
    for key in range(1, repeat):
        name = fake.name()
        r.set(key, name)
        sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
        val = (key, name)
        mycursor.execute(sql, val)
    t1 = time.time()
    ta = ta + (t1 - t0)
    r.flushall()
    mycursor.execute("DROP TABLE tablica")
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)


Average write time after repeating the benchmark 100 times is:  0.430205500125885


In [ ]:
#Benchmarking how long it takes to read data without a cache once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    sql = "SELECT value FROM tablica WHERE id = %s"
    tpl = (key, )
    mycursor.execute(sql, tpl)
    myresult = mycursor.fetchall()
    print(myresult)
t1 = time.time()
print("Time it took to write: ", t1-t0)
mycursor.execute("DROP TABLE tablica")

[('Jennifer Wilson',)]
[('Lauren Williams',)]
[('Timothy Mitchell',)]
[('Rebecca Perez',)]
[('Christopher Bradley',)]
[('Brandon Cook',)]
[('David Wall',)]
[('Angela Rodgers',)]
[('Mary Lambert',)]
[('Christopher Ellis',)]
[('Patricia Thomas',)]
[('Paul Wong',)]
[('Kaitlin Evans',)]
[('Michael Simmons',)]
[('Katie Williams',)]
[('Alicia Martinez',)]
[('Robert Bell DDS',)]
[('Jamie Bird',)]
[('James Medina',)]
[('Victoria Rodriguez',)]
[('Travis Yu',)]
[('Shannon Kent',)]
[('Theresa Richards',)]
[('Brian Shepard',)]
[('Andrew Montgomery',)]
[('Jamie Hampton',)]
[('Frank Fields',)]
[('Amanda Lara',)]
[('William Walters',)]
[('Katelyn Brown',)]
[('Anita Herrera',)]
[('Chad Hooper',)]
[('Michael Hill',)]
[('Rebekah Valenzuela',)]
[('Kathryn Alexander',)]
[('Richard Garcia',)]
[('Ryan Harris',)]
[('Mr. Justin Nash',)]
[('Eugene Walker',)]
[('Judith Bolton',)]
[('Cameron Wright',)]
[('Tonya Campos',)]
[('Jeffery Walsh',)]
[('Chad Miller DDS',)]
[('Tiffany Wells',)]
[('Sandra Larsen',)]
[('Er

In [ ]:
#Benchmarking how long it takes to read data without a cache 100 times and calculating the average read time
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average write time after repeating the benchmark 100 times is: ", ta)
mycursor.execute("DROP TABLE tablica")

[('Evelyn Ramos',)]
[('Alex Cox',)]
[('Lisa Deleon',)]
[('Anthony Munoz',)]
[('Michael Oconnor',)]
[('Matthew Herrera',)]
[('Emily Watkins',)]
[('Samantha Wilkins',)]
[('Pamela Ramirez',)]
[('Courtney Warren',)]
[('Catherine Maldonado',)]
[('Bridget Mueller',)]
[('Laura Garcia',)]
[('William Davis',)]
[('James Richard',)]
[('Brendan Smith',)]
[('Andrew Horton',)]
[('Caitlyn Schultz',)]
[('Chris Thompson',)]
[('Amanda Sanchez',)]
[('Amy Adams',)]
[('Joshua Pena',)]
[('Danielle Kelly',)]
[('Dr. James Young',)]
[('Brianna Harris',)]
[('Mallory Reed',)]
[('Theresa Robbins',)]
[('Jamie Manning',)]
[('Marcus Pearson',)]
[('Kathryn Bray',)]
[('Peter Wong',)]
[('Alexandria Miller',)]
[('Megan Wilson',)]
[('Jennifer Harris',)]
[('Brittany Frye',)]
[('Dana Koch',)]
[('Laura Reynolds',)]
[('Nicholas Murphy',)]
[('Emily Soto',)]
[('David Trevino',)]
[('Cody Butler',)]
[('Elizabeth Gonzalez',)]
[('Victoria Coleman',)]
[('Andrew Vaughn',)]
[('Katherine Rodriguez',)]
[('Robert Flores',)]
[('John Lewi

In [ ]:
#Benchmarking reads with cache hit rate 0% once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to read data: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

[('Claudia Lara',)]
[('Jonathan Wells',)]
[('Elijah Powers',)]
[('Richard Banks',)]
[('Steven Moore',)]
[('Jacob Hall',)]
[('Gina Carroll',)]
[('Sherry Campbell',)]
[('Austin Henry',)]
[('Steven Tucker',)]
[('Raymond Fletcher',)]
[('Spencer Medina',)]
[('Robert Smith',)]
[('Julie Mccoy',)]
[('Joel Jordan',)]
[('Tracy James',)]
[('Scott Miller',)]
[('Greg Boyd',)]
[('Rebecca Jacobs',)]
[('John Green',)]
[('David Allen',)]
[('Wendy Russell',)]
[('Joe Cohen',)]
[('Kyle Lopez',)]
[('Angela Flores',)]
[('Tony Brooks',)]
[('Rebecca Hinton',)]
[('April Wallace',)]
[('Joe Thomas',)]
[('Nicole Reynolds',)]
[('Tanya Martin',)]
[('Robert Williams',)]
[('Melanie Roy',)]
[('Eric Hood',)]
[('Michael Sutton',)]
[('Julia Perez',)]
[('Katherine Roberts',)]
[('Marvin Paul',)]
[('John Bailey',)]
[('Edward Turner DVM',)]
[('Melissa Boyd',)]
[('Jeremy Smith',)]
[('Danielle Rogers',)]
[('Jordan Holland',)]
[('Robert Smith',)]
[('Aaron Zamora',)]
[('Gerald Meadows',)]
[('Samantha Hull MD',)]
[('Paul Blevins'

In [ ]:
#Benchmarking reads with cache hit rate 0% 100 times and calculating average time it takes to execute reads
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

[('Phillip Bailey',)]
[('Patrick Stevens',)]
[('Susan Perez',)]
[('Scott Hogan',)]
[('Vernon Lam',)]
[('Daisy Turner',)]
[('Nicholas Cervantes',)]
[('David Rivas',)]
[('Kathleen Adams',)]
[('Monique Woods',)]
[('Deborah Wilcox',)]
[('Alicia House',)]
[('William Campbell',)]
[('Veronica Martinez',)]
[('Alvin Jenkins',)]
[('Daniel Johnson',)]
[('Douglas Robinson',)]
[('Michele Fowler',)]
[('Tonya Parker DDS',)]
[('Brian Reyes',)]
[('Shawna Dawson',)]
[('Amy Sanchez',)]
[('Ruben Hunter',)]
[('Mitchell Dixon',)]
[('Richard Blevins',)]
[('Holly Allen',)]
[('Amber Marquez',)]
[('Robert Jenkins',)]
[('Nicole Hoffman',)]
[('Catherine Brooks',)]
[('Jonathan Ortiz',)]
[('Darryl Collins',)]
[('David Hanson',)]
[('Ebony Lucero',)]
[('Margaret Hoffman',)]
[('Nicholas Cain',)]
[('Maurice Welch',)]
[('Anthony Quinn',)]
[('Kenneth Sanford',)]
[('Marc Johnson',)]
[('Mark Daniels',)]
[('Bailey Osborn',)]
[('Paul Case',)]
[('Richard Rodriguez',)]
[('Jordan Barber',)]
[('Christopher Valdez',)]
[('Michelle

In [ ]:
#Benchmarking reads with cache hit rate 50% once
cutoff = repeat * 0.5
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to write: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Thomas Rodriguez
James Schmitt
Melinda Hernandez
Michelle Sanchez
Amanda Acosta
Mark Roberts
Michelle Freeman
Cynthia Wilson
Casey Baldwin
Kyle Turner
Diana Carter
Lisa Gould
Amy Silva
Brenda Henry
Margaret Gonzales
David Atkins
Teresa Arnold
Wesley Sexton
Victoria Lewis
Brandi Perez
James Long
Rachel Douglas
Brandi Hernandez
James Lewis
Willie Noble
Brent Mckenzie
Megan Brooks PhD
Mason Barnes
Ronnie Barnett
Elijah Bell
Terri Stark
Tina Figueroa
Chelsea Carson
Elizabeth Shannon
Gloria Davis
Logan Poole
Kelly Love
Stephen Diaz
Edward Robertson
Bryan Matthews
Daniel Stokes
Matthew Richardson
Vincent King
Kenneth Johnson
Ryan Holmes
John Turner
Laura Hammond
Erin Barnett
Wendy Campbell
[('Paul Hernandez',)]
[('Benjamin Craig',)]
[('Latoya Carroll',)]
[('Robin Barajas',)]
[('Aaron Contreras',)]
[('Laura Alexander',)]
[('Yolanda Hunt',)]
[('Jessica Sparks',)]
[('Dillon Cantrell',)]
[('Sherri Pena',)]
[('Richard Owens',)]
[('Mrs. Tracy Aguirre',)]
[('Antonio Alexander',)]
[('Jason Bautista'

In [ ]:
#Benchmarking reads with cache hit rate 50% 100 times and calculating average read time
cutoff = repeat * 0.5
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Michelle Wang
Anthony Watkins
Paul Martinez
Zachary Brooks
John Turner
Derek Pearson
Fred Collins
Megan Werner
Melanie Villarreal
Donna Long
Thomas Russell
Linda Foster
John Noble
Cheryl Bowers
Summer Nichols
Ryan Hooper MD
Alexander Raymond
Jeffrey Lee
Sharon Myers
Brenda Walker
Jane Anderson
Amber Sanders
Michael Munoz
Amy Martin
Kenneth Chavez
Caroline Nguyen
Kenneth Edwards
Benjamin Johnson
Alan Dixon
Kim Sanchez
Donald Smith
Deborah Yates
Laura Murray
Elaine Thomas
Amanda Jackson
Luke Rodriguez
Hannah Maxwell
Christopher Baker
Gregory Reeves
Justin Mcdaniel
Gregory Reed
Mr. John Bailey
Victor Mitchell
Kristen Mitchell
Jeffrey Estrada
Tyler Cruz
Albert Matthews
Briana Stuart
Zachary Bond
[('Jacob Strickland',)]
[('Diane Anderson',)]
[('Mary Poole',)]
[('Sonya Young',)]
[('Alan Lam',)]
[('Catherine Faulkner',)]
[('Julie Nunez',)]
[('Adrian Hughes',)]
[('John Anderson',)]
[('Melissa Mitchell',)]
[('William Davis',)]
[('William Moody',)]
[('Michael Green',)]
[('Mrs. Amanda Cohen MD',)

In [ ]:
#Benchmarking reads with cache hit rate 80% once
cutoff = repeat * 0.8
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to read: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Debbie Collins
Derek Mendez
Nancy Crawford
Theresa Moore
Stanley Navarro
Christopher Hall
Austin Romero
Karen Rodriguez
Kara Jackson MD
Michael Adams
Dr. Jennifer Maddox
Carol Henry
Mary Church
Vincent Mccarty
Laurie Russell
Molly Johnson
David Mcmillan
Phillip Cook
Steven Meyer
Jonathan Ellis
Mary Hanson
Jamie Hammond
John Crawford
Bethany Wilson
Vanessa Peters
Christopher Rubio
Brandon Salazar
Joseph Warren
Jennifer Cortez
Kristopher Tucker
Tracy Carpenter
Joshua Arias
Darrell Reed
Eric Wang
Jeffrey Romero
Chad Martin
Tina Smith
Matthew Bush
Stephanie Solomon
James Stephenson
Jo Coffey
Anthony Gross
Dominic Hodges
Kathryn Ponce
James Jones
Tracey Bell
Matthew Jones
Timothy Elliott
Tammy Walker
Ashley Robinson
Rhonda Vazquez
Matthew Nelson
Mitchell Sawyer
Stephen Sims
Jasmine Bauer
Edward Kennedy
Marissa Mendez
Jessica Dunn
Jeffrey Carson
Matthew Craig
Michael Salinas
Jennifer Moore
Andrea Padilla
Steve Fischer
Maria Walker
Margaret Parker
Gregory Knight
Lawrence Phillips
Robert Avery

In [ ]:
#Benchmarking reads with cache hit rate 80% 100 times and calculating average read time
cutoff = repeat * 0.8
cutoff = int(cutoff)
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, cutoff):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
for key in range(cutoff, repeat):
    name = fake.name()
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

David Wilson
Jessica Crawford
Ryan Hill
Robin Gould
Teresa Gould
Andrea Crawford
Amanda Lewis
Paul Gonzalez
David Alvarado
Anna Forbes
Denise Meadows
Crystal Reed
Michelle Hutchinson
Lauren Weeks
Michael Gilmore
Christopher Clark
John Webb
Kelly Wilson
Nancy Willis
Keith Cruz
Joshua Flowers
Duane Butler
Sydney Perry
Monica Mcpherson
Austin Garcia
Megan Lopez
Kevin Lewis
Isaiah Lozano
Rebecca Hill
David Davis
Scott Jones
Timothy Hendricks
James Atkinson
Kelly Brown
Alexander Green
Michael Richardson
Perry Bauer
Dustin Wagner
Connie Smith
Mark Rodriguez
Steven Sullivan
Stanley Brewer
Christopher Thomas
Justin Myers
Leslie Jackson
Robert Wolf Jr.
Andrew Hamilton
Kimberly Thornton
Andrew Bradford
William Poole
Richard Bailey
Jessica Martin
Mr. Noah Mccormick
Donna Harris
Luis Crawford
Richard Acosta
Sherry Miller
Benjamin Snow
Andrew Velazquez
Fernando Nicholson
Edward Hall
Jonathan Marks
Megan Edwards
David Fox
Abigail Reynolds
Faith Vargas
Rebekah Flores
Kimberly Owens
Jerry Oliver
Virgi

In [ ]:
#Benchmarking reads with cache hit rate 100% once
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
t0 = time.time()
for key in range(1, repeat):
    temp = r.get(key)
    if temp == None:
        sql = "SELECT value FROM tablica WHERE id = %s"
        tpl = (key, )
        mycursor.execute(sql, tpl)
        myresult = mycursor.fetchall()
        print(myresult)
    else:
        print(temp)
t1 = time.time()
print("Time it took to write: ", t1-t0)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Ashley Foster
Kelly Stephens
Cynthia Bryan
Patricia Fitzpatrick
Sharon Black
David Floyd
David Ortiz
Erik Cox
David Stevens
Gina Foster
Michael Cannon
Jamie Johnson
Eddie Allison
Priscilla Becker
Patricia Cochran
Joseph Olson
Kent Castillo
Krista Dyer
Jaime Johnson
Taylor Kaiser
Gregory Howard
Jennifer Contreras
Rebecca Clark
Francisco Miller
Ryan Bush
Eric Hurst
Nicholas Tran
Michael Kemp
Joshua Golden
Nicole Johnson
Susan White
Phillip Hamilton
Sean Rodriguez
Aaron Jones
Jeff Wright
Glenn White
Jody Owens
Brandon Thomas
Brian Austin
Anthony Herring
Teresa Taylor
Shelby Williams
Johnathan Colon
Pamela Phillips
John Anderson
Leslie Bell
Michelle Alvarez
Charlene Porter
Douglas Blackwell
Mr. Christopher Roberts
Mr. Peter Jenkins
Kelly Hines
Erica Walker
Maria Campbell
Ronald Bauer
Alexander Jenkins
Jacob Yang
Daniel Miles
Courtney Holmes
Nathan Harris
Kimberly Hubbard
Heather Wallace
Lisa Lawson
Sarah Miller
Brittany Mendoza
Robyn Spears
Laura Navarro
Sarah Brown
Jeffrey Fletcher
Willia

In [ ]:
#Benchmarking reads with cache hit rate 100% 100 times and calculating average read time
mycursor.execute("CREATE TABLE tablica (id MEDIUMINT NOT NULL, value VARCHAR(50) NOT NULL, PRIMARY KEY (id))")
for key in range(1, repeat):
    name = fake.name()
    r.set(key, name)
    sql = "INSERT INTO tablica (id, value) VALUES (%s, %s)"
    val = (key, name)
    mycursor.execute(sql, val)
ta = 0
for i in range(100):
    t0 = time.time()
    for key in range(1, repeat):
        temp = r.get(key)
        if temp == None:
            sql = "SELECT value FROM tablica WHERE id = %s"
            tpl = (key, )
            mycursor.execute(sql, tpl)
            myresult = mycursor.fetchall()
            print(myresult)
        else:
            print(temp)
    t1 = time.time()
    ta = ta + (t1 - t0)
ta = ta/100
print("Average read time after repeating the benchmark 100 times is: ", ta)
r.flushall()
mycursor.execute("DROP TABLE tablica")

Carrie Johnson
Audrey Johnson
Ann Hamilton
Ricardo Blake
Christina Kennedy
Timothy Davis
Damon Parsons
Maria Davidson
Steve Greene
Michael Austin
Jacob Patel
Christopher Russell
James Wang
Jonathan Vazquez
Nancy Arnold
Caroline Zamora
Keith Price
Maria Garcia
Julie Melendez
Jennifer Monroe
Mark Mcintosh
Stacey Hughes
Scott Sanders
Alexandra Hernandez
Sandra Duncan
Michelle Johnson
Jamie Thomas
Emily Holt
Maria Robinson
Robert Campos
Latasha Sims
Allison Mitchell
Emily Sullivan
Wanda Beltran
Cheyenne Castillo
Joy Williams
Gregory Hawkins
Ronald Harding
Jake Mcbride
William Williamson
Kevin Romero
Melvin Mcguire
Kyle Byrd
Krista Moore
Samantha Black
Heather Palmer
Kristopher Crawford
Jacqueline Meyers
Kari Howard
Jonathan Scott
Stephen Wyatt
Edward Thomas
Jacob Greene
Tabitha Williams
Lisa Clayton
Bryan Robinson
Robert Davis
Sheryl Austin
Tyler Parrish
Alyssa Porter
Lori Nunez
Andrea Johns
Amber Wong MD
Megan Carter
Teresa Salinas
Donna Davis
Kelly Snow
Shawn Hodges
Cameron Alvarado
Alex

In [ ]:
!docker compose down

[+] Running 0/0
 ⠋ Container redis-server  Stopping                                        0.1s 
 ⠋ Container ipvo-db-1     Stopping                                        0.1s 
[+] Running 0/2
 ⠙ Container redis-server  Stopping                                        0.2s 
 ⠙ Container ipvo-db-1     Stopping                                        0.2s 
[+] Running 0/2
 ⠹ Container redis-server  Stopping                                        0.3s 
 ⠹ Container ipvo-db-1     Stopping                                        0.3s 
[+] Running 0/2
 ⠸ Container redis-server  Stopping                                        0.4s 
 ⠸ Container ipvo-db-1     Stopping                                        0.4s 
[+] Running 0/2
 ⠼ Container redis-server  Stopping                                        0.5s 
 ⠼ Container ipvo-db-1     Stopping                                        0.5s 
[+] Running 1/2
 ✔ Container redis-server  Removed                                         0.5s 
 ⠴ Container 